In [1]:
import os
import openai
from dotenv import load_dotenv
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By

In [2]:
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
driver = webdriver.Chrome('drivers/chromedriver108')

# Global Context

In [4]:
base = 'https://demo.saleor.io/default-channel/en-US/'

In [5]:
driver.get(base)

In [6]:
doc = bs(driver.find_element(By.TAG_NAME, 'html').get_attribute('innerHTML'), 'html.parser')

In [7]:
title = str(doc.head.title) + '\n'
metas = '\n'.join(list(map(lambda x: str(x), doc.head.find_all('meta'))))
head_string = title + metas

In [9]:
category_prompt = '''The following text is the url and content of head tag of a website.
url: {0}
head: {1}
The website category is:'''.format(base, head_string)

In [10]:
category_prompt

'The following text is the url and content of head tag of a website.\nurl: https://demo.saleor.io/default-channel/en-US/\nhead: <title>Saleor React Storefront</title>\n<meta charset="utf-8"/>\n<meta content="width=device-width" name="viewport"/>\n<meta content="index,follow" name="robots"/>\n<meta content="Saleor React Storefront" property="og:title"/>\n<meta content="https://reactstorefront.vercel.app/default-channel/en-US/" property="og:url"/>\n<meta content="https://og-image.vercel.app/React%20Storefront.png?theme=light&amp;md=1&amp;fontSize=100px&amp;images=https%3A%2F%2Fassets.vercel.com%2Fimage%2Fupload%2Ffront%2Fassets%2Fdesign%2Fvercel-triangle-black.svg&amp;images=https%3A%2F%2Fsaleor.io%2Fstatic%2Flogo-ad1b99aa7c6f5acf58a61640af760cfd.svg" property="og:image"/>\n<meta content="Hero image" property="og:image:alt"/>\n<meta content="2048" property="og:image:width"/>\n<meta content="1170" property="og:image:height"/>\n<meta content="Saleor React Storefront" property="og:site_name

In [11]:
category_res = openai.Completion.create(
    model="text-davinci-003",
    prompt=category_prompt,
    temperature=0,
    top_p=1,
    frequency_penalty=0.0,
    presence_penalty=0.0,
    max_tokens=256
)

category_res

<OpenAIObject text_completion id=cmpl-6fuQsY1krIHxXTNgH9zX10v5fvQBw at 0x10d5d2540> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": " eCommerce"
    }
  ],
  "created": 1675446394,
  "id": "cmpl-6fuQsY1krIHxXTNgH9zX10v5fvQBw",
  "model": "text-davinci-003",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 2,
    "prompt_tokens": 383,
    "total_tokens": 385
  }
}

In [12]:
global_context = 'The website name is {0}. This website is a {1} website.'.format(
    driver.current_url,
    category_res.choices[0].text.strip()
)

global_context

'The website name is https://demo.saleor.io/default-channel/en-US/. This website is a eCommerce website.'

# Page Context

In [13]:
driver.get('https://demo.saleor.io/default-channel/en-US/account/register/')

In [14]:
body_doc = bs(driver.find_element(By.TAG_NAME, 'body').get_attribute('innerHTML'), 'html.parser')

In [15]:
headings = []
for h in range(6):
    headings = [*headings, *list(map(lambda x: x.get_text().strip(), body_doc.find_all('h' + str(h + 1))))]

In [16]:
local_prompt = '''{0}
We are in page {1} of the website. The headings in this page are: {2}.
The page main functionality is'''.format(
    global_context,
    driver.current_url,
    ', '.join(headings)
)

print(local_prompt)

The website name is https://demo.saleor.io/default-channel/en-US/. This website is a eCommerce website.
We are in page https://demo.saleor.io/default-channel/en-US/account/register/ of the website. The headings in this page are: Create a new account.
The page main functionality is


In [17]:
local_res = openai.Completion.create(
    model="text-davinci-003",
    prompt=local_prompt,
    temperature=0,
    top_p=1,
    frequency_penalty=0.0,
    presence_penalty=0.0,
    max_tokens=256,
    stop=['.']
)

print(local_res.choices[0].text)

 to allow users to create a new account on the website


In [18]:
local_context = '''{0}. We are in page {1}. The page main functionality is {2}.'''.format(
    global_context,
    driver.current_url,
    local_res.choices[0].text.strip()
)

local_context

'The website name is https://demo.saleor.io/default-channel/en-US/. This website is a eCommerce website.. We are in page https://demo.saleor.io/default-channel/en-US/account/register/. The page main functionality is to allow users to create a new account on the website.'

# Input Context

In [19]:
print(body_doc.find_all('form')[0])

<form method="post"><div><h1 class="text-2xl font-bold">Create a new account</h1></div><div class="my-3"><label class="block text-md mb-2" for="email">Email</label><input class="px-4 w-full border-2 py-2 rounded-md text-sm outline-none" id="email" name="email" spellcheck="false" type="email"/></div><div class="mt-5"><label class="block text-md mb-2" for="password">Password</label><input class="px-4 w-full border-2 py-2 rounded-md text-sm outline-none" id="password" name="password" spellcheck="false" type="password"/></div><div class=""><button class="mt-4 mb-3 w-full bg-green-500 hover:bg-green-400 text-white py-2 rounded-md transition duration-100" type="submit">Register</button></div></form>


In [21]:
input_prompt = '''{0}
We want to complete the form on this page.
The input on the page has label {1}, and its html is {2}. Provide a value for the input:'''.format(
    local_context,
    'password',
    body_doc.find_all('form')[0].find_all('input')[1]
)

input_prompt

'The website name is https://demo.saleor.io/default-channel/en-US/. This website is a eCommerce website.. We are in page https://demo.saleor.io/default-channel/en-US/account/register/. The page main functionality is to allow users to create a new account on the website.\nWe want to complete the form on this page.\nThe input on the page has label password, and its html is <input class="px-4 w-full border-2 py-2 rounded-md text-sm outline-none" id="password" name="password" spellcheck="false" type="password"/>. Provide a value for the input:'

In [22]:
input_res = openai.Completion.create(
    model="text-davinci-003",
    prompt=input_prompt,
    temperature=0.7,
    top_p=1,
    frequency_penalty=0.0,
    presence_penalty=0.0,
    max_tokens=256,
)

print(input_res.choices[0].text.strip())

MyPassword123!


# Error Context

In [31]:
sample_error = '''Your password must be all in lower case.
Your password must not contain any special characters.'''

In [32]:
values = [
    "MyPassword123!",
    "mypassword123!",
    "MyPassword123"
]

values_string = '"' + '", "'.join(values)+ '"'

values_string

'"MyPassword123!", "mypassword123!", "MyPassword123"'

In [33]:
error_prompt = '''{0}
The input on the page has label {1}, and its html is {2}.
We entered the following values for input: {3} and we got the following error: "{4}".
A valid input sample is:'''.format(
    local_context,
    'password',
    body_doc.find_all('form')[0].find_all('input')[1],
    values_string,
    sample_error
)

error_prompt


'The website name is https://demo.saleor.io/default-channel/en-US/. This website is a eCommerce website.. We are in page https://demo.saleor.io/default-channel/en-US/account/register/. The page main functionality is to allow users to create a new account on the website.\nThe input on the page has label password, and its html is <input class="px-4 w-full border-2 py-2 rounded-md text-sm outline-none" id="password" name="password" spellcheck="false" type="password"/>.\nWe entered the following values for input: "MyPassword123!", "mypassword123!", "MyPassword123" and we got the following error: "Your password must be all in lower case.\nYour password must not contain any special characters.".\nA valid input sample is:'

In [34]:
error_res = openai.Completion.create(
    model="text-davinci-003",
    prompt=error_prompt,
    temperature=0,
    top_p=1,
    frequency_penalty=0.0,
    presence_penalty=0.0,
    max_tokens=256,
)

print(error_res.choices[0].text.strip())

"mypassword123".
